<a href="https://colab.research.google.com/github/GAL1LA0/Demo_Testing/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning notebook
inspired by Fiebig, Lennart; [youtube tutorial](https://www.youtube.com/watch?v=nsdCRVuprDY); [hugging face](https://huggingface.co/learn/nlp-course/chapter7/3?fw=pt)

*   Scrapping from wikipedia for some content for our model. We are scrapping a statical website therefore the scraper is built like the following:
For more information regarding dynamical webscrapping please refer to webscraer notebook



In [ ]:
import requests
import bs4

response = requests.get('https://de.wikipedia.org/wiki/SAP_S/4HANA',headers={'User-Agent': 'Mozilla/5.0'})# deutsch englisch egal?
soup = bs4.BeautifulSoup(response.text,features="html.parser")

#soup.body.get_text(' ', strip=True)
generated_Text = soup.body.get_text(' ', strip=True)
print(generated_Text)

SAP S/4HANA aus Wikipedia, der freien Enzyklopädie Zur Navigation springen Zur Suche springen SAP S/4HANA Basisdaten Entwickler SAP SE Erscheinungsjahr 2015 Aktuelle Version 2023 [1] ( 11. Oktober 2023 ) Betriebssystem Linux Programmiersprache ABAP , C Kategorie ERP Lizenz proprietäre Lizenz deutschsprachig ja Produktseite SAP S/4HANA ist eine ERP -Softwarelösung der SAP SE und Nachfolger des bisherigen Kernprodukts SAP ECC (SAP E RP C entral C omponent). Das S steht dabei für simple oder suite , die 4 für die vierte Produktgeneration und SAP HANA ( H igh Performance An alytic A ppliance) für die zugrunde liegende Datenbanktechnologie. SAP S/4HANA ist das favorisierte Produkt seitens SAP für den Einsatz in einer Cloud. Inhaltsverzeichnis 1 Verwendung 2 Geschichte 3 Versionen 4 Übersicht der Release-Stände (On-Premise und Cloud) 5 Integration in das Unternehmen 6 Implementierung 7 Addons von Drittanbietern 8 Produktlebenszyklus 9 Wartungsende 10 Einzelnachweise Verwendung [ Bearbeiten |



*   First try with gpt2. Here I tried to fine-tune the gpt2 model with the help of masked language model (AutoModelForMaskedLM) approach. However, this approach only works with certain models, such as [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased).




In [ ]:
from transformers import GPT2Tokenizer, AutoModelForMaskedLM

# Instantiate the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize input text
input_text = generated_Text
tokens = tokenizer(input_text, return_tensors="pt")

# Instantiate the pre-trained GPT-2 model for masked language modeling
model = AutoModelForMaskedLM.from_pretrained("gpt2")

# Forward pass with the tokenized input through the model
outputs = model(**tokens)

# Access the model's output
logits = outputs.logits


Token indices sequence length is longer than the specified maximum sequence length for this model (5846 > 1024). Running this sequence through the model will result in indexing errors


ValueError: ignored

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model_checkpoint = "gpt2"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

ValueError: ignored

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

*   Second try with gpt2 model: In this approach we try to train the model with our scraped text using [hugging face](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer) Trainer-class
*   At first we importe the necessery packages, prepare the toknizer and specify the model we want to use





In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer.save_pretrained("/content/gpt2_tokenizer")

# Confirm that the tokenizer is saved in the Colab environment
!ls /content/gpt2_tokenizer

config.json		merges.txt	   special_tokens_map.json  vocab.json
generation_config.json	model.safetensors  tokenizer_config.json


In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.save_pretrained("/content/gpt2_tokenizer")
!ls /content/gpt2_tokenizer

config.json		merges.txt	   pytorch_model.bin	    tokenizer_config.json
generation_config.json	model.safetensors  special_tokens_map.json  vocab.json


*   trying to generate a text before training the model and observing the result


In [ ]:
output = model.generate(
    tokenizer.encode("Das S steht dabei", return_tensors='pt'),
    do_sample=True,
    max_length = 30,
    pad_token_id = model.config.eos_token_id,
    top_k=50,
    top_p=0.95,
)
print(tokenizer.decode(output[0],skip_special_tokens=True))

Das S steht dabei, saal aieleit hir wert daat daat saam, nee kut




*   putting the text in a txt file for the further usage/simulating a training corpus
*  Utilizing the load_dataset and load_data_collator to load/prepare the dataset for training. These methods are going to be used in the train method, which is responsible for the actual training of the model



In [ ]:
file_name = "/content/wikipedia.txt"

with open(file_name, "w") as file:
    file.write(generated_Text)

!ls /content
#from google.colab import files
#files.download(file_name)

cached_lm_GPT2Tokenizer_128_wikipedia.txt	gpt2_tokenizer	sample_data
cached_lm_GPT2Tokenizer_128_wikipedia.txt.lock	output_dir	wikipedia.txt


In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip show accelerate

Name: accelerate
Version: 0.25.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
train(
    train_file_path='/content/wikipedia.txt',
    model_name='gpt2',
    output_dir='/content/output_dir',
    overwrite_output_dir=False,
    per_device_train_batch_size=8,
    num_train_epochs=50.0,
    save_steps=50000
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: ignored

## **logits, etc below are for mlm - next step?** - can be ignored. maybe useful for the further development

In [ ]:
import torch
#def Text_spliter(Text_webscraper):
  # Split the text into chunks
chunk_size = 512
text_chunks = [input_text[i:i + chunk_size] for i in range(0, len(input_text), chunk_size)]

  # Tokenize each chunk separately
all_tokens = []
for chunk in text_chunks:
  tokens = tokenizer(chunk, return_tensors="pt", truncation=True)
  all_tokens.append(tokens)

  # Concatenate the tokenized chunks
combined_tokens = {key: torch.cat([t[key] for t in all_tokens], dim=1) for key in all_tokens[0].keys()}

  # Forward pass with the tokenized input through the model
outputs = model(**combined_tokens)
  #return outputs

In [ ]:
tokens = tokenizer(generated_Text, return_tensors="pt")
output = model(**tokens)
logits = outputs.logits

In [ ]:
from transformers import GPT2Tokenizer, AutoModelForMaskedLM
import torch

# Instantiate the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = AutoModelForMaskedLM.from_pretrained("gpt2")

# Your long text
long_text = generated_Text

# Tokenize the long text
tokens = tokenizer(long_text, return_tensors="pt", truncation=True)

# Extract the input_ids from the tokens
input_ids = tokens["input_ids"]

# Define the chunk size
chunk_size = 512

# Split the input_ids into chunks
chunks = [input_ids[:, i:i+chunk_size] for i in range(0, input_ids.shape[1], chunk_size)]

# Forward pass through the model for each chunk
for idx, chunk in enumerate(chunks):
    # Perform the forward pass
    outputs = model(**{"input_ids": chunk})

    # Access the model's output (adjust this based on your specific model and task)
    logits = outputs.logits

    # You can do further processing with the logits or other model outputs

    # Print information about the chunk and its output
    print(f"Chunk {idx} length: {chunk.shape[1]}")
    print(f"Logits shape: {logits.shape}")

# Now you can use the model outputs as needed
